In [3]:
## Bibliotecas e conexão com o banco de dados
import pandas as pd
import sys
sys.path.append(r'C:\Users\giovanni.5683\GITHUB\controlling_postgreSQL')  # Adiciona o diretório onde está conn.py
from conn import get_connection
engine = get_connection()
cursor = engine.cursor()

In [5]:
query_byd = cursor.execute("""
SELECT * FROM byd.bonus_view
""")
byd = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
byd

,idnfsexterno,revenda,empresa,numero_nota_fiscal,serie_nota_fiscal,dta_processamento,tipo_transacao,des_tipo_transacao,cod_cliente,nome_cliente,...,opt_comecial,campanha,detalhes,bonus_utilizado,bonus_dpto,rebate_dpto,ipva_dpto,trade_mkt_dpto,data_final_venda,competencia
0,A25519901V210,5,25,1990,1,2025-10-13 17:04:15,V21,VENDA DE VEICULOS NOVOS,1642684,AUTOPLUS COMERCIO DE VEICULOS LTDA,...,Não se Aplica,None,BÔNUS EMPLACAMENTO,Equalização,14221.4,None,None,None,2025-10-13 17:04:15,Outubro
1,A25252731V210,2,25,5273,1,2025-10-13 17:06:36,V21,VENDA DE VEICULOS NOVOS,1613600,AUTOPLUS COMERCIO DE VEICULOS LTDA,...,Não se Aplica,None,BÔNUS EMPLACAMENTO,Equalização,14221.4,None,None,None,2025-10-13 17:06:36,Outubro
2,A251250881V210,1,25,25088,1,2025-09-29 08:19:09,V21,VENDA DE VEICULOS NOVOS,3773712,ROBERTO KOVALEZUK,...,Não se Aplica,198/25,197/25 BÔNUS EMPLACAMENTO,Equalização,9221.4,None,None,None,2025-09-29 08:19:09,Setembro
3,A2583641V210,8,25,364,1,2025-10-15 10:32:25,V21,VENDA DE VEICULOS NOVOS,3630549,AUTOPLUS COMERCIO DE VEICULOS LTDA,...,Opção 1,197/25 - 214/25,BÔNUS EMPLACAMENTO,Equalização,14221.4,None,None,None,2025-10-15 10:32:25,Outubro
4,A251214891V210,1,25,21489,1,2025-07-17 13:59:33,V21,VENDA DE VEICULOS NOVOS,36945,AUTOPLUS COMERCIO DE VEICULOS LTDA,...,None,None,None,Varejo,None,None,None,None,2025-07-17 13:59:33,Julho
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4750,A25525221V210,5,25,2522,1,2025-11-29 19:15:13,V21,VENDA DE VEICULOS NOVOS,3839588,SANDRO COUTO,...,Não se Aplica,055/2025,BLACK,"Varejo, Trade Marketing",None,None,None,20000.0,2025-11-29 19:15:13,Novembro
4751,A25343781V210,3,25,4378,1,2025-11-29 15:01:02,V21,VENDA DE VEICULOS NOVOS,3830664,FERNANDA DA CRUZ DIANA,...,Não se Aplica,224/25,None,Sem Bônus,None,None,None,None,2025-11-29 15:01:02,Novembro
4752,A25344101V210,3,25,4410,1,2025-11-29 19:39:34,V21,VENDA DE VEICULOS NOVOS,3839427,LORELEY MOTTER KIKUTI,...,Não se Aplica,None,None,Sem Bônus,None,None,None,None,2025-11-29 19:39:34,Novembro
4753,A2584951V210,8,25,495,1,2025-11-26 11:48:52,V21,VENDA DE VEICULOS NOVOS,692337,J R FAVRETTO ADM IMOBILIARIA LTDA,...,Opção 1,224/2025,None,Taxa 0,None,None,None,None,2025-11-26 11:48:52,Novembro


In [6]:
# Verificar a estrutura da bonus_view
query_structure = cursor.execute("""
SELECT column_name, data_type 
FROM information_schema.columns 
WHERE table_schema = 'byd' AND table_name = 'bonus_view'
ORDER BY ordinal_position
""")
structure = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
structure

,column_name,data_type
0,idnfsexterno,text
1,revenda,text
2,empresa,text
3,numero_nota_fiscal,text
4,serie_nota_fiscal,text
...,...,...
64,rebate_dpto,text
65,ipva_dpto,text
66,trade_mkt_dpto,text
67,data_final_venda,timestamp without time zone


In [7]:
# Verificar valores distintos de bonus_utilizado
query_bonus = cursor.execute("""
SELECT DISTINCT bonus_utilizado, COUNT(*) as quantidade
FROM byd.bonus_view
GROUP BY bonus_utilizado
ORDER BY quantidade DESC
""")
bonus_status = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
bonus_status

,bonus_utilizado,quantidade
0,Sem Bônus,1353
1,Varejo,1031
2,Taxa 0,433
3,Trade_in,379
4,Trade Marketing,182
5,"Varejo, Trade Marketing",178
6,Venda Direta,166
7,Veículo de Devolução,162
8,Refaturado,157
9,"Taxa 0, Trade_in",154


In [8]:
# Criar a view pendente_verificacao_view
create_view = """
CREATE OR REPLACE VIEW byd.pendente_verificacao_view AS
SELECT *
FROM byd.bonus_view
WHERE bonus_utilizado = 'PENDENTE VERIFICACAO'
ORDER BY dta_processamento DESC;
"""

cursor.execute(create_view)
engine.commit()
print("✓ View 'pendente_verificacao_view' criada com sucesso!")

✓ View 'pendente_verificacao_view' criada com sucesso!


In [9]:
# Testar a nova view
query_pendente = cursor.execute("""
SELECT * FROM byd.pendente_verificacao_view
""")
pendente_df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
print(f"Total de registros pendentes: {len(pendente_df)}")
pendente_df.head(10)

Total de registros pendentes: 30


,idnfsexterno,revenda,empresa,numero_nota_fiscal,serie_nota_fiscal,dta_processamento,tipo_transacao,des_tipo_transacao,cod_cliente,nome_cliente,...,opt_comecial,campanha,detalhes,bonus_utilizado,bonus_dpto,rebate_dpto,ipva_dpto,trade_mkt_dpto,data_final_venda,competencia
0,A25420191V210,4,25,2019,1,2025-12-11 17:22:50,V21,VENDA DE VEICULOS NOVOS,3853400,LUIZ HENRIQUE OGG,...,None,None,None,PENDENTE VERIFICACAO,None,None,None,None,2025-12-11 17:22:50,Dezembro
1,A25345351V210,3,25,4535,1,2025-12-11 15:27:25,V21,VENDA DE VEICULOS NOVOS,3852474,CAMILA FERNANDES,...,None,None,None,PENDENTE VERIFICACAO,None,None,None,None,2025-12-11 15:27:25,Dezembro
2,A25345341V210,3,25,4534,1,2025-12-11 15:27:02,V21,VENDA DE VEICULOS NOVOS,3511025,ALINE LETICIA SOARES DE MELO,...,None,None,None,PENDENTE VERIFICACAO,None,None,None,None,2025-12-11 15:27:02,Dezembro
3,A25420171V210,4,25,2017,1,2025-12-11 14:22:04,V21,VENDA DE VEICULOS NOVOS,623813,IOLANDA SILVA DA MAIA,...,None,None,None,PENDENTE VERIFICACAO,None,None,None,None,2025-12-11 14:22:04,Dezembro
4,A25345301V210,3,25,4530,1,2025-12-11 14:16:23,V21,VENDA DE VEICULOS NOVOS,3831120,JESUS DANIEL GARCIA MARCANO,...,None,None,None,PENDENTE VERIFICACAO,None,None,None,None,2025-12-11 14:16:23,Dezembro
5,A25526581V210,5,25,2658,1,2025-12-11 14:03:03,V21,VENDA DE VEICULOS NOVOS,3764223,RUBIA JEANE ALESSIO,...,None,None,None,PENDENTE VERIFICACAO,None,None,None,None,2025-12-11 14:03:03,Dezembro
6,A25345281V210,3,25,4528,1,2025-12-11 13:46:21,V21,VENDA DE VEICULOS NOVOS,3849346,ALEX SANTANA DO NASCIMENTO,...,None,None,None,PENDENTE VERIFICACAO,None,None,None,None,2025-12-11 13:46:21,Dezembro
7,A25526571V210,5,25,2657,1,2025-12-11 13:38:38,V21,VENDA DE VEICULOS NOVOS,834922,FLAMARION DE BARROS CORDEIRO,...,None,None,None,PENDENTE VERIFICACAO,None,None,None,None,2025-12-11 13:38:38,Dezembro
8,A25345261V210,3,25,4526,1,2025-12-11 12:15:50,V21,VENDA DE VEICULOS NOVOS,3852098,ROJAS DA COSTA SERVICOS MEDICOS LTDA,...,None,None,None,PENDENTE VERIFICACAO,None,None,None,None,2025-12-11 12:15:50,Dezembro
9,A25420161V210,4,25,2016,1,2025-12-11 11:40:06,V21,VENDA DE VEICULOS NOVOS,231294,PAULO BOAVA DA SILVA,...,None,None,None,PENDENTE VERIFICACAO,None,None,None,None,2025-12-11 11:40:06,Dezembro


## ✓ View Pendente Verificação Criada

A view `byd.pendente_verificacao_view` foi criada com sucesso!

**Características:**
- Filtra automaticamente registros com `bonus_utilizado = 'PENDENTE VERIFICACAO'`
- Possui as mesmas 69 colunas da `bonus_view`
- **Atualização automática**: Como é uma VIEW (não tabela materializada), ela sempre reflete os dados mais recentes da `bonus_view` em tempo real
- Sempre que um registro for adicionado, removido ou modificado na `bonus_view`, a `pendente_verificacao_view` mostrará as mudanças instantaneamente

**Como usar:**
```sql
SELECT * FROM byd.pendente_verificacao_view;
```